# MARA Graph

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.colors as mcolors
from matplotlib.cm import Blues

In [ ]:
def parse_strategies_from_log(log_file):
    """Extract defense strategies and attack paths from the log file"""
    defense_strategy = {}
    attack_paths = []
    attack_probs = []
    
    try:
        with open(log_file, 'r') as f:
            log_content = f.read()
            
        # Parse attack paths
        attack_paths_section = log_content.split('Debug - Current Attack paths:')[1].split('++++++')[0].strip()
        for line in attack_paths_section.strip().split('\n'):
            if ':' in line:
                path_index, path_str = line.strip().split(':', 1)
                attack_paths.append(path_str.strip())
        
        # Parse defense strategy
        defense_section = log_content.split('optimal defense strategy:')[1].split('worst case attack strategies')[0]
        for line in defense_section.strip().split('\n')[1:]:  # Skip header
            if line.strip():
                parts = line.strip().split()
                if len(parts) >= 2:
                    node_id = int(parts[0])
                    probability = float(parts[1])
                    defense_strategy[node_id] = probability
        
        # Parse attack strategy probabilities
        attack_section = log_content.split('worst case attack strategies per goal:')[1].split('[1]')[0]
        for line in attack_section.strip().split('\n')[1:]:  # Skip header
            if line.strip():
                parts = line.strip().split()
                if len(parts) >= 2:
                    prob = float(parts[1])
                    attack_probs.append(prob)
                    
        return defense_strategy, attack_paths, attack_probs
    
    except Exception as e:
        print(f"Error parsing log file: {e}")
        return {}, [], []

In [ ]:
def defense_color_mapping(probability, min_prob=0.05, max_prob=1.0):
    """Map defense probability to a blue gradient color"""
    # Normalize the probability to a value between 0 and 1
    if max_prob == min_prob:
        normalized = 1.0  # Avoid division by zero
    else:
        normalized = (probability - min_prob) / (max_prob - min_prob)
    
    # CHANGE THIS LINE:
    # From: color_value = 0.3 + (normalized * 0.6)
    # To: color_value = 0.5 + (normalized * 0.4)
    color_value = 0.65 + (normalized * 0.4)
    return Blues(color_value)

In [ ]:
def graph_plotting():
    """Enhanced graph plotting with strategy information from log file"""
    basename = 'experiment_1'
    
    # Parse strategies from log file
    defense_strategy, attack_paths, attack_probs = parse_strategies_from_log(f"{basename}.log")
    
    # Filter defense strategies with probabilities < 5%
    filtered_defense = {node: prob for node, prob in defense_strategy.items() if prob >= 0.05}
    
    # Find min and max defense probabilities for gradient scaling
    if filtered_defense:
        min_prob = min(filtered_defense.values())
        max_prob = max(filtered_defense.values())
    else:
        min_prob, max_prob = 0.05, 1.0
    
    # Create a basic spring layout as starting point
    pos = nx.spring_layout(attack_graph, k=1, iterations=50, seed=42)
    
    # Identify key nodes (using actual node numbers in the graph)
    initial_node = 1  # Node 1 (green, entry node)
    target_nodes = [6, 9]  # Nodes 6 and 9 (red, target nodes)
    
    # Identify intermediate nodes
    all_nodes = list(attack_graph.nodes())
    intermediate_nodes = [node for node in all_nodes if node != initial_node and node not in target_nodes]
    
    # Calculate node distances from initial node (shortest path length)
    distances = {}
    for node in all_nodes:
        try:
            distances[node] = nx.shortest_path_length(attack_graph, initial_node, node)
        except nx.NetworkXNoPath:
            # No path from initial to this node, set a large distance
            distances[node] = len(attack_graph)
    
    # Normalize distances to determine y-coordinate
    max_dist = max(distances.values()) if distances else 1
    
    # Set y-coordinate based on distance from initial node
    for node in all_nodes:
        dist = distances.get(node, 0)
        # Nodes farther from initial node go lower on the graph
        pos[node][1] = 1.0 - (dist / max_dist if max_dist > 0 else 0)
    
    # Ensure initial node is at the top and target nodes at the bottom
    pos[initial_node][1] = 1.0  # Place at the top
    
    # Place target nodes at the bottom
    for node in target_nodes:
        if node in pos:  # Make sure node exists
            pos[node][1] = 0.0  # Place at the bottom
    
    # Adjust x-coordinates of target nodes to spread them horizontally
    if len(target_nodes) > 1:
        x_values = [pos[node][0] for node in intermediate_nodes if node in pos]
        min_x, max_x = min(x_values), max(x_values) if x_values else (-0.5, 0.5)
        spread = max_x - min_x
        
        # Spread target nodes horizontally
        if target_nodes[0] in pos:
            pos[target_nodes[0]][0] = min_x - 0.1 * spread if spread > 0 else -0.3
        if target_nodes[1] in pos:
            pos[target_nodes[1]][0] = max_x + 0.1 * spread if spread > 0 else 0.3
    
    # Center the initial node horizontally
    if initial_node in pos:
        pos[initial_node][0] = 0

    pos[2][0] = -0.2  # Move node 2 more to the left/center
    
    plt.figure(figsize=(10, 6))

    # Draw edges with arrows
    nx.draw_networkx_edges(attack_graph, pos, 
                        edge_color='gray',
                        arrows=True,
                        arrowsize=20,
                        width=2)

    # Create color map for nodes
    node_colors = []
    for node in attack_graph.nodes():
        if node == 1:  # Node 1 (initial node)
            node_colors.append('lightgreen')
        elif node == 6 or node == 9:  # Target nodes
            node_colors.append('lightcoral')
        else:  # Intermediate nodes
            node_colors.append('lightblue')
    
    # Create a list to track which nodes need white text (darker nodes)
    white_text_nodes = []
    
    # Create a mapping between node list index and actual node number
    node_list = list(attack_graph.nodes())
    
    # Apply defense gradient coloring
    for node, prob in filtered_defense.items():
        if node in node_list:
            idx = node_list.index(node)
            node_colors[idx] = defense_color_mapping(prob, min_prob, max_prob)
            # If the node color is dark enough, use white text
            if prob > (min_prob + max_prob) / 2:
                white_text_nodes.append(node)
    
    # Draw nodes
    nx.draw_networkx_nodes(attack_graph, pos,
                        node_color=node_colors,
                        node_size=700,
                        edgecolors='black',
                        linewidths=2)

    # Create label dictionary using actual node numbers
    node_labels = {node: str(node) for node in attack_graph.nodes()}

    # Draw all labels in black
    nx.draw_networkx_labels(attack_graph, pos, 
                        node_labels, 
                        font_size=12,
                        font_weight='bold',
                        font_color='black')
    
    # Then overlay white labels for dark nodes
    white_labels = {node: str(node) for node in white_text_nodes if node in node_labels}
    if white_labels:
        nx.draw_networkx_labels(attack_graph, pos, 
                            white_labels, 
                            font_size=12,
                            font_weight='bold',
                            font_color='white')

    # Create legend showing optimal defense strategies and attack paths
    legend_elements = []
    
    # Add defense strategies to legend with gradient colors
    sorted_defense = sorted(filtered_defense.items(), key=lambda x: x[1], reverse=True)
    for node, prob in sorted_defense:
        color = defense_color_mapping(prob, min_prob, max_prob)
        legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', 
                                         label=f'Defense: Node {node} ({prob:.2f})',
                                         markerfacecolor=color, markersize=10))
    
    # Add attack paths to legend
    for i, (path, prob) in enumerate(zip(attack_paths, attack_probs)):
        if prob >= 0.05:  # Only show attack paths with ≥5% probability
            legend_elements.append(plt.Line2D([0], [0], marker='', color='black', linestyle='-',
                                             label=f'Attack path {i}: {path} with weight {prob:.2f}'))
    
    # Add legend to the plot
    plt.legend(handles=legend_elements, title="Optimal Strategies (≥5%)", 
              loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=2)

    #plt.title("Attack Graph with Defender Strategy Gradient", pad=20, fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
graph_plotting()